In [1]:
from faster_whisper import WhisperModel
from autogen_ext.models.ollama import OllamaChatCompletionClient
from autogen_agentchat.agents import AssistantAgent
from pydantic import BaseModel
from typing import List, Optional

ModuleNotFoundError: No module named 'faster_whisper'

In [2]:
import torch
print("PyTorch version:", torch.__version__) 

PyTorch version: 2.6.0+cu118


In [3]:


def transcribe_audio_with_timestamps(audio_path: str) -> str: # ---> To pass audio file
    """
    Transcribes the audio file with timestamps using Whisper.
    """
    model = WhisperModel("tiny", device="cpu", compute_type="int8")  # or tiny/small/medium/large
    result = model.transcribe(audio_path) # ---> prints progress + timestamps while running in the notebook/console 
    
    sum_text = ""
    segments, info = model.transcribe(audio_path, beam_size=5)
    for segment in segments:
        sum_text += segment.text + " "
        print("[%.2fs -> %.2fs] %s" % (segment.start, segment.end, segment.text))
    return sum_text

In [51]:
transcription_text = transcribe_audio_with_timestamps(r"C:\Users\divya\OneDrive\Desktop\Dungeon\dd.mp3")

[0.00s -> 5.28s]  Dungeons & Dragons is like a board game without the board and you play using your imagination.
[5.28s -> 8.00s]  If you think of something, it can just happen in game.
[8.00s -> 12.32s]  D&D is very popular, almost everyone I've ever met absolutely loves it,
[12.32s -> 16.08s]  and I have never played it before, even though I always thought I would really like it.
[16.08s -> 18.48s]  People would tell me stories of their games like,
[18.48s -> 21.28s]  so this one time me and my party were entering a cave,
[21.28s -> 23.76s]  and there was a big ogre guarding some treasure,
[23.76s -> 26.16s]  but I used my charisma stat to seduce him,
[26.16s -> 27.84s]  and we ended up aloping together.
[27.84s -> 30.48s]  Dude, if you don't think that sounds crazy fun,
[30.48s -> 33.44s]  then I don't think we would get along, and you should probably answer to crime.
[34.56s -> 37.04s]  But if you do think that sounds fun, then hey!
[37.84s -> 39.92s]  Sorry, I've never done that b

In [91]:
from pydantic import BaseModel
from typing import List, Optional

class StoryEvent(BaseModel):
    event: str
    details: str

class PlayerAction(BaseModel):
    player_placeholder: str       # e.g., Player1, Player2
    character_name: str           # actual in-game character name
    action: str
    outcome: str

class WorldStateUpdate(BaseModel):
    aspect: str
    change: str

class CharacterTracking(BaseModel):
    character_name: str           # in-game character name
    condition: str
    notes: Optional[str] = ""

class UnresolvedThread(BaseModel):
    mystery_or_quest: str
    status: str

class DungeonSessionSummary(BaseModel):
    story_progression: List[StoryEvent]
    player_actions: List[PlayerAction]
    world_state_updates: List[WorldStateUpdate]
    character_tracking: List[CharacterTracking]
    unresolved_threads: List[UnresolvedThread]


In [92]:
model_client = OllamaChatCompletionClient(
    model="deepseek-r1:14b",
    name="D&D Summarizer",
    response_format= DungeonSessionSummary,
    model_info = {
          "vision": False,
          "function_calling": True,
          "json_output": True,
    },
    system_message=( """
You are a Dungeon Master session summarizer.
Your job is to take the raw transcript of a Dungeons & Dragons game 
and output a structured summary in CSV format only (no extra text).

The CSV must contain five sections, each separated by a header row:

1. Story Progression
   Columns: Event,Details

2. Player Actions
   Columns: Player,Action,Outcome

3. World State Updates
   Columns: Aspect,Chang

4. Character Tracking
   Columns: Character,Condition,Notes

5. Unresolved Threads / Hooks
   Columns: Mystery/Quest,Status

Rules:
- Only output valid CSV (commas as separators).
- Always include the header row for each section.
- If a section has no data, include only the header row.
- No markdown, no explanations, only plain CSV.
- Replace real player names with placeholders (Player1, Player2, etc.).
- Replace any personal or sensitive information with generic terms.
- Output only valid CSV as before.
"""

    ),
)


In [93]:
summarizer = AssistantAgent(
    name="Dungeon_Scribe_Summarizer",
    model_client=model_client,
    description="An AI assistant that summarizes tabletop RPG session transcripts and tracks story developments."
)

In [94]:
result = await summarizer.run(task=f"Summarize this session:\n{transcription_text}")
result

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content="Summarize this session:\n Dungeons & Dragons is like a board game without the board and you play using your imagination.  If you think of something, it can just happen in game.  D&D is very popular, almost everyone I've ever met absolutely loves it,  and I have never played it before, even though I always thought I would really like it.  People would tell me stories of their games like,  so this one time me and my party were entering a cave,  and there was a big ogre guarding some treasure,  but I used my charisma stat to seduce him,  and we ended up aloping together.  Dude, if you don't think that sounds crazy fun,  then I don't think we would get along, and you should probably answer to crime.  But if you do think that sounds fun, then hey!  Sorry, I've never done that before, it felt weird.  I decided to reach out to Charlie, aka Slimesicle, aka Charlie Slimesicle,  because he's super into D&D, 

In [95]:
csv_output = result.messages[-1].content.strip()

In [96]:

with open("dnd_summary_deepseek-r114b", "w", encoding="utf-8") as f:
    f.write(csv_output)
print("✅ Summary saved to dnd_summary.csv")

✅ Summary saved to dnd_summary.csv
